In [ ]:
!pip install langchain openai
!pip install --upgrade langchain langchain-community openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.8/464.8 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Fo

In [ ]:
# Import necessary libraries
from transformers import pipeline  # For using Hugging Face models
from langchain.llms.base import LLM  # Base class for LangChain LLMs
from transformers import AutoTokenizer  # For tokenizing input text
import json  # For loading the product catalog from a JSON file
import logging  # For suppressing unnecessary logs

# Suppress logs from the transformers library to avoid clutter
logging.getLogger("transformers").setLevel(logging.ERROR)


In [ ]:
#open the Json file that includes the product catalog
with open('product_catalog.json','r') as file:
    product_catalog = json.load(file)

# Print the product catalog
print(product_catalog)
products = product_catalog["products"]

{'products': [{'id': 1, 'name': 'Wireless Earbuds', 'price': 15.99, 'category': 'Electronics', 'description': 'High-quality wireless earbuds with noise cancellation.', 'delivery_charge': 5.0, 'availability': True}, {'id': 2, 'name': 'Bluetooth Speaker', 'price': 25.99, 'category': 'Electronics', 'description': 'Portable Bluetooth speaker with 10-hour battery life.', 'delivery_charge': 7.0, 'availability': True}, {'id': 3, 'name': 'Coffee Maker', 'price': 45.99, 'category': 'Home Appliances', 'description': 'Automatic coffee maker with programmable settings.', 'delivery_charge': 10.0, 'availability': False}, {'id': 4, 'name': 'Running Shoes', 'price': 19.99, 'category': 'Sports', 'description': 'Lightweight running shoes with cushioned soles.', 'delivery_charge': 5.0, 'availability': True}, {'id': 5, 'name': 'Smart Watch', 'price': 99.99, 'category': 'Electronics', 'description': 'Smart watch with heart rate monitoring.', 'delivery_charge': 8.0, 'availability': True}, {'id': 6, 'name': 

In [ ]:
# Define a custom LLM class that integrates with LangChain
class HuggingFaceLLM(LLM):
    def _call(self, prompt: str, stop=None) -> str:
        """
        Generate a response using the Hugging Face GPT-2 model.
        Args:
            prompt (str): The user's query.
            stop (optional): Tokens to stop generation. Defaults to None.
        Returns:
            str: The generated response.
        """
        # Load the tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained("gpt2", max_length=512, truncation=True)
        chatbot = pipeline("text-generation", model="gpt2",tokenizer=tokenizer,framework="tf")
        # Generate a response with a maximum of 50 new tokens
        response = chatbot(prompt, max_new_tokens=50)
        return response[0]['generated_text']

    @property
    def _llm_type(self) -> str:
        """
        Return the type of LLM being used.
        """
        return "huggingface-gpt2"

# Create an instance of the custom LLM
huggingface_llm = HuggingFaceLLM()

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import openai


# Set your OpenAI API key
openai.api_key = "your-api-key"

# Initialize memory for multi-turn conversation
memory = ConversationBufferMemory()

# Initialize LLM with OpenAI's GPT-4
llm = ChatOpenAI(model_name="gpt-4", openai_api_key=openai.api_key)

# Create a conversation chain with memory
conversation = ConversationChain(
    llm=llm,
    memory=memory
)



<ipython-input-5-39d2bd358452>:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-5-39d2bd358452>:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", openai_api_key=openai.api_key)
<ipython-input-5-39d2bd358452>:17: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation =

In [ ]:

# Filter products based on the query
def filter_products(query, products):
    """
    Filter products based on the user's query.
    Args:
        query (str): The user's query.
        products (list): The list of products to filter.
    Returns:
        list or str: Filtered products or a specific response (e.g., delivery charge).
    """
    query = query.lower()  # Make the query case-insensitive

    # Store query in memory
    memory.save_context({"input": query}, {"output": "Processing query..."})

    # Retrieve past conversation history
    past_conversation = memory.buffer
   # print("Memory buffer:", past_conversation)  # Debugging: See stored memory

    # Check if the user asks a follow-up question
    if "which one" in query or "what about" in query or "any other" in query:
        last_query = past_conversation.split("\n")[-2]  # Get last query
        #print("Previous query:", last_query)  # Debugging

        # If last query was about a category or price, repeat the search
        return filter_products(last_query, products)

   # last_product=memory.load_memory_variables({})

    # Handling follow-up question: "How much does it cost?"

   # Handling follow-up question: "How much does it cost?"
    if "how much does it cost" in query or "price" in query:
        # Retrieve the last product mentioned from memory
        last_product = memory.load_memory_variables({}).get("history", "")
        if last_product:
            # Extract the last product name from the conversation history
            for p in products:
                if p["name"].lower() in last_product.lower():
                    return f"{p['name']} costs ${p['price']}."
        return "Please specify a product name."

    # Handle price-based queries (e.g., "under $20")
    if "under" in query and "$" in query:
        price_limit = int(query.split("under")[1].strip().replace("$", ""))
        return [p for p in products if p["price"] < price_limit]

    # Handle category-based queries (e.g., "find me electronics")
    elif "category" in query or any(word in query for word in ["electronics", "sports", "home appliances", "accessories"]):
        category = None
        if "electronics" in query:
            category = "Electronics"
        elif "sports" in query:
            category = "Sports"
        elif "home appliances" in query:
            category = "Home Appliances"
        elif "accessories" in query:
            category = "Accessories"
        if category:
            return [p for p in products if p["category"].lower() == category.lower()]

    # Handle availability-based queries (e.g., "show me available products")
    elif "available" in query or "in stock" in query:
        return [p for p in products if p["availability"]]

    # Handle delivery charge queries (e.g., "what is the delivery charge for this product?")
    elif "delivery charge" in query or "delivery cost" in query:
        # Extract the product name more robustly
        product_name = None
        if "for" in query:
            product_name = query.split("for")[1].strip()
        elif "of" in query:
            product_name = query.split("of")[1].strip()

        if product_name:
            # Remove any trailing punctuation or question marks
            product_name = product_name.rstrip("?.,")

            # Find the product in the catalog
            for p in products:
                if p["name"].lower() == product_name.lower():
                    return f"The delivery charge for {p['name']} is ${p['delivery_charge']}."

        return "Please specify a product to check the delivery charge."

    # Default: Return all products if no specific query is matched
    return products


In [ ]:
# Chatbot function
def chatbot(query, products):
    """
    Generate a response to the user's query.
    Args:
        query (str): The user's query.
        products (list): The list of products to filter.
    Returns:
        str: The chatbot's response.
    """
    filtered_products = filter_products(query, products)

    # Handle delivery charge queries separately
    if isinstance(filtered_products, str):
        return filtered_products

    # Create a structured response
    if not filtered_products:
        return "No products found matching your query."

    response = f"Here are the products based on your query:\n"
    for product in filtered_products:
        response += f"- {product['name']} (${product['price']}, {product['category']}, Delivery: ${product['delivery_charge']}, Available: {product['availability']})\n"

    return response

In [ ]:
# Test the chatbot
print(chatbot("Find me all products under $20", products))
print(chatbot("Show me all electronics", products))
print(chatbot("What is the delivery charge for Wireless Earbuds?", products))
print(chatbot("How much does it cost", products))
print(chatbot("Show me available products", products))


Here are the products based on your query:
- Wireless Earbuds ($15.99, Electronics, Delivery: $5.0, Available: True)
- Running Shoes ($19.99, Sports, Delivery: $5.0, Available: True)
- Yoga Mat ($12.99, Sports, Delivery: $4.0, Available: True)
- Wireless Mouse ($9.99, Electronics, Delivery: $3.0, Available: True)
- Water Bottle ($8.99, Accessories, Delivery: $3.0, Available: True)

Here are the products based on your query:
- Wireless Earbuds ($15.99, Electronics, Delivery: $5.0, Available: True)
- Bluetooth Speaker ($25.99, Electronics, Delivery: $7.0, Available: True)
- Smart Watch ($99.99, Electronics, Delivery: $8.0, Available: True)
- Wireless Mouse ($9.99, Electronics, Delivery: $3.0, Available: True)
- Gaming Keyboard ($49.99, Electronics, Delivery: $8.0, Available: True)
- Fitness Tracker ($59.99, Electronics, Delivery: $6.0, Available: True)

The delivery charge for Wireless Earbuds is $5.0.
Wireless Earbuds costs $15.99.
Here are the products based on your query:
- Wireless E

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output ,Markdown


In [ ]:
# Create an input box for user queries
input_box = widgets.Text(
    placeholder="Type your message...",
    description="You:",
    layout=widgets.Layout(width="80%")
)

# Create an output area where chatbot responses will appear
output_box = widgets.Output()

In [ ]:
def chatbot_interaction(change):
    user_input = input_box.value.strip()
    input_box.value = ""  # Clear input box

    response = chatbot(user_input, products)  # Get chatbot response

    with output_box:
        output_box.clear_output()  # Clear previous output
        print(f"You: {user_input}")
        display(Markdown(f"**Bot:**\n\n{response}"))  # Format response nicely

input_box.on_submit(chatbot_interaction)
display(input_box, output_box)

Text(value='', description='You:', layout=Layout(width='80%'), placeholder='Type your message...')

Output()